In [1]:
import pandas as pd
import numpy as np
import urllib.request, json
from pandas.io.json import json_normalize
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [4]:
# Загрузить данные из файлов
matches706_team_picks = pd.read_csv('table/table wtih team & picks path 7.06.csv').drop(['Unnamed: 0'], axis=1)
matches707_team_picks = pd.read_csv('table/table wtih team & picks path 7.07 to 29.12.17.csv', index_col=0)

# новая нумерация индексов
matches706_team_picks = matches706_team_picks.reset_index().drop(['index'], axis=1)
matches707_team_picks = matches707_team_picks.reset_index().drop(['index'], axis=1)

# Соединить даныне
matches_team_picks = matches706_team_picks.append(matches707_team_picks)

#Разделение данных без перемешивания на traning & test для каждого патча и для соединенных данных
X_706, y_706 = np.split(matches706_team_picks, [len(matches706_team_picks) - int(len(matches706_team_picks)*0.3)]) 
X_707, y_707 = np.split(matches707_team_picks, [len(matches707_team_picks) - int(len(matches707_team_picks)*0.3)]) 
X, y = np.split(matches_team_picks, [len(matches_team_picks) - 400]) 

X_train = X.drop(['radiant_win'], axis=1)
y_train = X['radiant_win']

X_test = y.drop(['radiant_win'], axis=1)
y_test = y['radiant_win']

X_train_706 = X_706.drop(['radiant_win'], axis=1)
y_train_706 = X_706['radiant_win']

X_test_706 = y_706.drop(['radiant_win'], axis=1)
y_test_706 = y_706['radiant_win']

X_train_707 = X_707.drop(['radiant_win'], axis=1)
y_train_707 = X_707['radiant_win']

X_test_707 = y_707.drop(['radiant_win'], axis=1)
y_test_707 = y_707['radiant_win']
matches706_team_picks.head()

,radiant_id,dire_id,radiant1,radiant2,radiant3,radiant4,radiant5,dire1,dire2,dire3,dire4,dire5,radiant_win
0,70.0,59.0,97,112,106,66,104,75,18,28,65,70,1
1,59.0,70.0,28,110,29,95,11,32,92,54,74,55,0
2,29.0,23.0,37,51,55,49,48,64,16,77,88,76,1
3,23.0,29.0,16,8,87,34,33,64,55,28,95,25,0
4,18.0,33.0,71,55,92,46,93,27,7,20,10,17,1


In [45]:
# Загрузить данные из файлов
matches = pd.read_csv('table/table wtih team, kills from 1.09.13 to 22.01.18.csv', index_col=0)
matches = matches.drop(['radiant_id','radiant_name', 'dire_id', 'dire_name','kills'],axis=1).reindex()

X, y = np.split(matches, [len(matches) - 400]) 

X_train = X.drop(['radiant_win'], axis=1)
y_train = X['radiant_win']

X_test = y.drop(['radiant_win'], axis=1)
y_test = y['radiant_win']
len(matches)

14760

In [21]:
# Случайный лес с параметрами которые оказались лучшими на обучении патча7,06
forest_last = RandomForestClassifier(n_estimators=36, random_state=42, 
                                                        max_depth=14, max_features=8)
forest_last.fit(X_train, y_train)
print (forest_last.score(X_train, y_train))
print (forest_last.score(X_test, y_test))

# forest_last.fit(X_train_706, y_train_706)
# print (forest_last.score(X_train_706, y_train_706))
# print (forest_last.score(X_test_706, y_test_706))

# forest_last.fit(X_train_707, y_train_707)
# print (forest_last.score(X_train_707, y_train_707))
# print (forest_last.score(X_test_707, y_test_707))

0.970543175487
0.475


In [22]:
# Нейронные сети
mlp = MLPClassifier(solver='lbfgs',random_state=1, hidden_layer_sizes=[10, 10, 10])

mlp.fit(X_train, y_train)
print (mlp.score(X_train, y_train))
print (mlp.score(X_test, y_test))
# print ('----------------------------')
# mlp.fit(X_train_706, y_train_706)
# print (mlp.score(X_train_706, y_train_706))
# print (mlp.score(X_test_706, y_test_706))
# print ('----------------------------')
# mlp.fit(X_train_707, y_train_707)
# print (mlp.score(X_train_707, y_train_707))
# print (mlp.score(X_test_707, y_test_707))

0.513300835655
0.475


In [25]:
# Масштабирование данных
scaler = StandardScaler()
scaler.fit(X_train)
print(scaler.mean_)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# X_train_707_scaled = scaler.transform(X_train_707)
# X_test_707_scaled = scaler.transform(X_test_707)

[ 56.15752089  53.34735376  53.73572423  52.61706128  52.56922006
  59.35424791  55.00835655  54.66504178  52.45821727  52.27618384]


In [26]:
mlp.fit(X_train_scaled, y_train)
print (mlp.score(X_train_scaled, y_train))
print (mlp.score(X_test_scaled, y_test))

# mlp.fit(X_train_707_scaled, y_train_707)
# print (mlp.score(X_train_707_scaled, y_train_707))
# print (mlp.score(X_test_707_scaled, y_test_707))

0.568593314763
0.4775
